# Automatic detection of the difficulty level of English-language films


**Цель исследования** - разработанная модель для автоматического определения уровня сложности англоязычных фильмов..

**Основные этапы исследования:**

- загрузка и ознакомление с данными,
- предварительная обработка,
- выбор и обучение модели,
- итоговая оценка качества предсказания.

Исходные данные:
- таблица с названием фильма и меткой уровня сложности,
- субтитры 278 англоязычных фильмов,
- Оксфордский словарь, содержащий разделенные по уровням лексические единицы.

## Обзор данных

In [1]:
RAND = 666 # константа случайного состояния

### Подключение необходимых библиотек

In [2]:
!pip install pysrt
!pip install spacy

In [3]:
import os
import numpy as np               
import pandas as pd              
import pysrt                     
import spacy                     
import re                        
from joblib import dump, load    



from sklearn.datasets import load_files  
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import ColumnTransformer

### Открытие и изучение данных

In [4]:
server_path = ''
local_path = ''

df = 'movies_labels.xlsx'
try:
    df = pd.read_excel(server_path + df, index_col='id')  
    
except: 
    df = pd.read_excel(local_path + df, index_col='id')  
    
df.head(10)

,Movie,Level
id,,
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+
5,All_dogs_go_to_heaven(1989),A2/A2+
6,An_American_tail(1986),A2/A2+
7,Babe(1995),A2/A2+
8,Back_to_the_future(1985),A2/A2+


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 240
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   241 non-null    object
 1   Level   241 non-null    object
dtypes: object(2)
memory usage: 5.6+ KB


Пропусков в таблице нет, посмотрим уникальные значения стобца с уровнем сложности.

In [6]:
df['Level'].unique()

array(['B1', 'B2', 'A2/A2+', 'C1', 'B1, B2', 'A2/A2+, B1', 'A2'],
      dtype=object)

In [7]:
df['Level'].value_counts()

B2            101
B1             55
C1             40
A2/A2+         26
B1, B2          8
A2              6
A2/A2+, B1      5
Name: Level, dtype: int64

Присвоим числовые метки вместо буквенных и объединим уровни 'A2', 'A2/A2+' и 'A2/A2+, B1'; 'B1' и 'B1, B2'. (Объединение субъективное и в любом случае повлияет на работу модели, но как лингвист утверждаю, что человек, владеющий уровнем А2 поймет 50-70% фильма уровня B1).

In [8]:
label_dict = {'A2': 1,
              'A2/A2+': 1,
              'B1': 2,
              'A2/A2+, B1': 1,
              'B2': 3,
              'B1, B2': 2,
              'C1': 4}

df = df.replace(label_dict)

In [9]:
df['Level'].value_counts()

3    101
2     63
4     40
1     37
Name: Level, dtype: int64

Удалим дубликаты.

In [10]:
df[df.duplicated(keep=False)]

,Movie,Level
id,,
38,Powder(1995),2
43,Inside_out(2015),2
44,Inside_out(2015),2
68,Powder(1995),2


In [11]:
df = df.drop_duplicates()
df.shape

(239, 2)

Подключим папку, содержащую файлы с субтитрами, и проверим сколько фильмов из датасета имеет и метку уровня сложности, и субтитры.

In [12]:
names = os.listdir(path='./subtitles')
print(f'Количество фильмов в папке с субтитрами: {len(names)}')

Количество фильмов в папке с субтитрами: 279


In [13]:
filtr = set(names) & set(df['Movie'] + '.srt')
print(f'Количество фильмов, имеющих метку и субтитры: {len(filtr)}')

Количество фильмов, имеющих метку и субтитры: 229


Загрузим словарь с уникальными лексемами и присвоенным уровнем сложности.

In [14]:
oxford = load_files('oxford/', shuffle=False, encoding='utf-8-sig')
oxford.data[0][:30]

'a, an \nabout \nabove \nacross \na'

## Подготовка данных для обучения модели

### Очистка текстовых данных.

In [15]:
HTML = r'<.*?>' 
TAG = r'{.*?}'
COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]' 
LETTERS = r'[^a-zA-Z\.,!? ]' 
SPACES = r'([ ])\1+' 
DOTS = r'[\.]+' 

def clean_subs(subs):
    subs = subs[1:] 
    txt = re.sub(HTML, ' ', subs.text)
    txt = re.sub(COMMENTS, ' ', txt) 
    txt = re.sub(LETTERS, ' ', txt) 
    txt = re.sub(DOTS, r'.', txt) 
    txt = re.sub(SPACES, r'\1', txt)
    txt = re.sub('www', '', txt) 
    txt = txt.lstrip() 
    txt = txt.encode('ascii', 'ignore').decode() 
    txt = txt.lower() 
    return txt


### Лемматизация и подсчет количесва лемм, содержащихся в словаре.

In [16]:
def lemma_count(lemmas, oxf, cat):
    func_dict = {'A1': 0,
                 'A2': 1,
                 'B1': 2,
                 'B2': 3,
                 'C1': 4}
    level = func_dict[cat]
    oxf_word_list = oxf[level].split()
    words = [lemma for lemma in lemmas if lemma in oxf_word_list]

    return len(set(words))

### Итоговый датасет.

In [17]:
for film in filtr:
    try: 
        subs = pysrt.open(f'subtitles/{film}')
    except:
        subs = pysrt.open(f'subtitles/{film}', encoding='iso-8859-1')
    
    cln_subs = clean_subs(subs)
    df.loc[df['Movie'] == film[:-4], 'subs'] = cln_subs
    
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(cln_subs)
    lemma_list = [token.lemma_ for token in doc]
    
    for lvl in ['A1', 'A2', 'B1', 'B2', 'C1']:
        df.loc[df['Movie'] == film[:-4], lvl+'_lemma_cnt'] = lemma_count(lemma_list, oxford.data, lvl)

df.head(10)

,Movie,Level,subs,A1_lemma_cnt,A2_lemma_cnt,B1_lemma_cnt,B2_lemma_cnt,C1_lemma_cnt
id,,,,,,,,
0,10_Cloverfield_lane(2016),2,"ben on phone michelle, please don t hang up. j...",372.0,187.0,128.0,105.0,39.0
1,10_things_I_hate_about_you(1999),2,"i ll be right with you. so, cameron. here you ...",449.0,243.0,176.0,161.0,68.0
2,A_knights_tale(2001),3,should we help him? he s due in the lists in t...,427.0,247.0,154.0,135.0,75.0
3,A_star_is_born(2018),3,get to it. black eyes open wide it s time to t...,455.0,257.0,143.0,127.0,48.0
4,Aladdin(1992),1,where the caravan camels roam where it s flat ...,424.0,246.0,155.0,149.0,64.0
5,All_dogs_go_to_heaven(1989),1,"itchy, a few more degrees to the left! now! ta...",376.0,204.0,107.0,116.0,40.0
6,An_American_tail(1986),1,"mama tanya, fievel? will you stop that twirlin...",364.0,151.0,95.0,73.0,27.0
7,Babe(1995),1,and how it changed our valley forever. there w...,410.0,227.0,126.0,114.0,48.0
8,Back_to_the_future(1985),1,"so right now, statler toyota is making the bes...",448.0,285.0,156.0,148.0,60.0


Проверим пропущенные значения.

In [18]:
df[df['subs'].isna()].sort_values('Movie')

,Movie,Level,subs,A1_lemma_cnt,A2_lemma_cnt,B1_lemma_cnt,B2_lemma_cnt,C1_lemma_cnt
id,,,,,,,,
237,Bullet train,2,NaN,NaN,NaN,NaN,NaN,NaN
235,Glass Onion,3,NaN,NaN,NaN,NaN,NaN,NaN
239,Lightyear,3,NaN,NaN,NaN,NaN,NaN,NaN
236,Matilda(2022),4,NaN,NaN,NaN,NaN,NaN,NaN
240,The Grinch,2,NaN,NaN,NaN,NaN,NaN,NaN
81,The Secret Life of Pets.en,3,NaN,NaN,NaN,NaN,NaN,NaN
238,Thor: love and thunder,3,NaN,NaN,NaN,NaN,NaN,NaN
106,Up (2009),1,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
sorted(set(df['Movie'] + '.srt') - set(names))

['Bullet train.srt',
 'Glass Onion.srt',
 'Lightyear.srt',
 'Matilda(2022).srt',
 'The Grinch.srt',
 'The Secret Life of Pets.en.srt',
 'Thor: love and thunder.srt',
 'Up (2009).srt']

In [20]:
df.dropna(inplace=True)
df[df['subs'].isna()]

,Movie,Level,subs,A1_lemma_cnt,A2_lemma_cnt,B1_lemma_cnt,B2_lemma_cnt,C1_lemma_cnt
id,,,,,,,,


Определим количество фильмов, представленных в каждой категории сложности.

In [21]:
df['Level'].value_counts()

3    97
2    59
4    39
1    36
Name: Level, dtype: int64

### Разбиение на обучающую и тестовую выборки.

In [22]:
df_train, df_test = train_test_split(df, random_state=RAND, test_size=.2, stratify=df['Level'])

X_train = df_train.drop(['Level', 'Movie'], axis=1)
y_train = df_train['Level']
X_test = df_test.drop(['Level', 'Movie'], axis=1)
y_test = df_test['Level']

print(f'Размер обучающей выборки: {X_train.shape, y_train.shape}')
print(f'Размер тестовой выборки: {X_test.shape, y_test.shape}')

Размер обучающей выборки: ((184, 6), (184,))
Размер тестовой выборки: ((47, 6), (47,))


### Векторизация текста субтитров.

In [23]:
vect = CountVectorizer(min_df=4).fit(X_train['subs'])
X_train_vect = vect.transform(X_train['subs'])
X_test_vect = vect.transform(X_test['subs'])

In [24]:
feature_names = vect.get_feature_names()
print('Количество признаков: {}'.format(len(feature_names)))
print('Каждый 100-й признак:\n{}'.format(feature_names[::100]))

Количество признаков: 8597
Каждый 100-й признак:
['aah', 'adorable', 'alley', 'anti', 'arrows', 'automatically', 'barn', 'belts', 'blinded', 'bowl', 'bubble', 'calling', 'caymans', 'cheese', 'cleaner', 'colour', 'conflict', 'core', 'crew', 'dandy', 'definition', 'devious', 'disrespectful', 'draft', 'earlier', 'emotion', 'eternity', 'explanation', 'fated', 'find', 'fool', 'fricking', 'genuine', 'grace', 'gumbo', 'hated', 'him', 'hour', 'impressive', 'instantly', 'island', 'journalist', 'knocked', 'leak', 'lining', 'loved', 'mandatory', 'meant', 'mind', 'most', 'nations', 'normally', 'one', 'oversee', 'passion', 'pet', 'planning', 'possible', 'printed', 'psst', 'rack', 'recognize', 'remote', 'rethink', 'role', 'sake', 'screams', 'services', 'shoots', 'sixteen', 'sneakers', 'speaking', 'started', 'strength', 'sum', 'switching', 'teeny', 'thinking', 'tokyo', 'travelled', 'typing', 'us', 'visit', 'wasting', 'whistle', 'workers']


## Обучение и предсказание модели Наивного Байеса.

In [25]:
other_colls = X_train.columns[1:]

vector = CountVectorizer(min_df=4)

preprocessor = ColumnTransformer([
    ('other', 'passthrough', other_colls),
    ('txt', vector, 'subs')
])

classifier = MultinomialNB()
pipe = Pipeline(steps=[('prep', preprocessor),
                       ('clf', classifier)])

param_grid = {'clf__alpha': np.arange(0.001, 0.3, 0.002)}
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_weighted')
grid.fit(X_train, y_train)

print(grid.best_score_)
print(grid.best_params_)

0.622562340320961
{'clf__alpha': 0.049}


Лучшая модель показала результат - 0.623 (F1_weighted) при подобранном гиперпараметре 'alpha' равном 0.049.

Оценим модель на тестовой выборке.

In [26]:
predictions = grid.predict(X_test)

print(f'Метрика F1-weighted на обучающей выборке:{f1_score(grid.predict(X_train), y_train, average="weighted")}')
print(f'Метрика F1-weighted на тестовой выборке:{f1_score(predictions, y_test, average="weighted")}')
print('-'*50)
print('Матрица ошибок:')
print('-'*50)
print(classification_report(y_test, predictions))


Метрика F1-weighted на обучающей выборке:0.9671595224794963
Метрика F1-weighted на тестовой выборке:0.5645276227944291
--------------------------------------------------
Матрица ошибок:
--------------------------------------------------
              precision    recall  f1-score   support

           1       0.57      0.57      0.57         7
           2       0.37      0.58      0.45        12
           3       0.72      0.65      0.68        20
           4       1.00      0.38      0.55         8

    accuracy                           0.57        47
   macro avg       0.67      0.54      0.56        47
weighted avg       0.66      0.57      0.58        47



Лучшая модель показала результат - 0.56 (F1_weighted) на тестовой выборке.

## Вывод

В целом с учетом предположения о субъективности разметки представленных Заказчиком фильмов, а так же использования быстрого и простого наивного Байесовского классификатора, удалось достичь приемлемых результатов. Взвешенная метрика F1-weighted на тестовой выборке составила 0.56.
